In [ ]:
# [예제 3.21] 인허가 엑셀 데이터를 read_excel로 읽어오기

import pandas as pd

# permission_path = r'C:\Users\JK\Desktop\부동산 데이터\주택건설인허가실적.xlsx'
permission_path = r'내려받은 인허가 엑셀파일의 위치\ 인허가 데이터 엑셀파일명.xlsx'
pd.read_excel(permission_path)

In [ ]:
# [예제 3.22] 인허가 엑셀 데이터를 read_excel로 읽어오기

permission_raw = pd.read_excel(permission_path, skiprows=10, index_col=0)

In [ ]:
# [예제 3.23] permission_df의 행과 열을 바꾸기

transposed_permission = permission_raw.T

In [ ]:
# [예제 3.24] 인덱스를 '연도.날짜' 형식으로 바꾸기

new_index = []

for old_date in transposed_permission.index:
    temp_list = old_date.split(' ')
    new_index.append(temp_list[0][:4] + '.' + temp_list[1][:2])

In [ ]:
# [예제 3.25] 인덱스를 새로 설정하고 데이터프레임 완성하기 

transposed_permission.index = pd.to_datetime(new_index)
transposed_permission.columns.name = None

In [ ]:
# [예제 3.26] 인허가 데이터를 데이터프레임으로 변환하는 함수 정의 

def permission_preprocessing(path):
    permission_raw = pd.read_excel(path, skiprows=10, index_col=0)
    transposed_permission = permission_raw.T
    new_index = []

    for old_date in transposed_permission.index:
        temp_list = old_date.split(' ')
        new_index.append(temp_list[0][:4] + '.' + temp_list[1][:2])
        
    transposed_permission.index = pd.to_datetime(new_index)
    transposed_permission.columns.name = None
    
    return transposed_permission

In [ ]:
# [예제 3.27] 미분양 엑셀 데이터를 read_excel로 읽어오기 

# unsold_path = r'C:\Users\JK\Desktop\부동산 데이터\시·군·구별+미분양현황_2082_128_20181229151931.xlsx'
unsold_path = r' 내려받은 미분양 데이터의 디렉터리 \ 미분양 데이터 엑셀파일명.xlsx'
unsold_raw = pd.read_excel(unsold_path, skiprows=1, index_col=0)

In [ ]:
# [예제 3.28] 미분양 데이터프레임 정리

del unsold_raw['시군구']
transposed_unsold = unsold_raw.T
transposed_unsold.index = pd.to_datetime(transposed_unsold.index)
transposed_unsold.columns.name = None

In [ ]:
# [예제 3.29] 미분양 데이터를 데이터프레임으로 변환하는 함수 정의 

def unsold_preprocessing(path):
    unsold_raw = pd.read_excel(path, skiprows=1, index_col=0)
    
    del unsold_raw['시군구']
    transposed_unsold = unsold_raw.T
    transposed_unsold.index = pd.to_datetime(transposed_unsold.index)
    transposed_unsold.columns.name = None
    
    return transposed_unsold

In [ ]:
# [예제 3.30] KBpriceindex_preprocessing 함수 가져오기

import xlwings as xw

def KBpriceindex_preprocessing(path, data_type):
    # path : KB 데이터 엑셀 파일의 디렉토리 (문자열)
    # data_type : ‘매매종합’, ‘매매APT’, ‘매매연립’, ‘매매단독’, ‘전세종합’, ‘전세APT’, ‘전세연립’, ‘전세단독’ 중 하나
    
    wb = xw.Book(path)                
    sheet = wb.sheets[data_type]   
    row_num = sheet.range(1,1).end('down').end('down').end('down').row  
    data_range = 'A2:GE' + str(row_num)
    raw_data = sheet[data_range].options(pd.DataFrame, index=False, header=True).value 
    
    bignames = '서울 대구 부산 대전 광주 인천 울산 세종 경기 강원 충북 충남 전북 전남 경북 경남 제주도 6개광역시 5개광역시 수도권 기타지방 구분 전국'
    bigname_list = bignames.split(' ')
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])

    for num, gu_data in enumerate(small_col):
        if gu_data == None:
            small_col[num] = big_col[num]

        check = num
        while True:
            if big_col[check] in bigname_list:
                big_col[num] = big_col[check]
                break
            else:
                check = check - 1
                
    big_col[129] = '경기' 
    big_col[130] = '경기'
    small_col[185] = '서귀포'
    
    raw_data.columns = [big_col, small_col]
    new_col_data = raw_data.drop([0,1])
    
    index_list = list(new_col_data['구분']['구분'])

    new_index = []

    for num, raw_index in enumerate(index_list):
        temp = str(raw_index).split('.')
        if int(temp[0]) > 12 :
            if len(temp[0]) == 2:
                new_index.append('19' + temp[0] + '.' + temp[1])
            else:
                new_index.append(temp[0] + '.' + temp[1])
        else:
            new_index.append(new_index[num-1].split('.')[0] + '.' + temp[0])

    new_col_data.set_index(pd.to_datetime(new_index), inplace=True)
    cleaned_data  = new_col_data.drop(('구분', '구분'), axis=1)
    return cleaned_data

In [ ]:
# [예제 3.31] 앞에서 정의한 함수들을 이용해 데이터 전처리하고 데이터프레임으로 가져오기

permission_path  = r'내려받은 인허가 엑셀파일의 위치\ 인허가엑셀 파일명.xlsx'
permission = permission_preprocessing(permission_path)
unsold_path = r' 내려받은 미분양 데이터의 디렉터리 \ 미분양 데이터 엑셀파일명.xlsx'
unsold = unsold_preprocessing(unsold_path)
kb_path = r' 여러분이 내려 받은 KB 엑셀파일의 디렉터리를 넣으세요 \ KB엑셀 파일명.xls'
price_index = KBpriceindex_preprocessing(kb_path, '매매종합')
jun_index   = KBpriceindex_preprocessing(kb_path, '전세종합')

In [ ]:
# [예제 3.32] 그래프를 위한 설정 (

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from matplotlib import style
style.use('ggplot')
%matplotlib inline

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
# 맥OS인 경우 위 두 줄을 입력하지 말고 아래 코드를 입력하세요
# rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# [예제 3.33] 서울의 인허가와 매매가 지수의 움직임을 그래프로 나타내기 

plt.figure(figsize=(10,6))
ax = plt.subplot()
ax2 = ax.twinx()

si = '서울'
gu = '서울'

plt.title(si + '-' + gu)
ln1 = ax.plot(price_index[si][gu]['2009-1':], label='매매가')
ln2 = ax2.plot(permission[si]['2009-1':], label='인허가', color='green',marker="o")
lns = ln1 +ln2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc='upper left')
    
plt.show()

In [ ]:
# [[예제 3.34] 인허가 데이터 연도별로 

year_permission = permission.groupby(permission.index.year).sum()

In [ ]:
# [예제 3.35] 인허가 데이터를 2년 뒤로 옮기기 

modified_permission = year_permission.shift(2)
temp = []
for year in modified_permission.index:
    temp.append(str(year) + '-6-1')
modified_permission.index = pd.to_datetime(temp)

In [ ]:
#[예제 3.36] 수정한 인허가 데이터와 매매가 지수 그래프 

plt.figure(figsize=(10,6))
ax = plt.subplot()
ax2 = ax.twinx()

si = '서울'
gu = '서울'

plt.title(si + '-' + gu)
ln1 = ax.plot(price_index[si][gu]['2009-1':], label='매매가')
ln2 = ax2.plot(modified_permission[si]['2009':], label='인허가', color='green',marker="o")
lns = ln1 +ln2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc='upper left')
    
plt.show()

In [ ]:
# [예제 3.37] 미분양 데이터를 추가한 그래프 

plt.figure(figsize=(10,6))
ax = plt.subplot()
ax2 = ax.twinx()

si = '서울'
gu = '서울'

plt.title(si + '-' + gu)
ln1 = ax.plot(price_index[si][gu]['2009-1':], label='매매가')
ln2 = ax.plot(jun_index[si][gu]['2009-1':], label='전세가')
ln3 = ax2.plot(modified_permission[si]['2009':]/10, label='인허가', color='lightslategray', ls='--')
ln4 = ax2.plot(unsold[si]['2009':], label='미분양', color='y', ls='--')
lns = ln1 +ln2 + ln3 + ln4
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc='upper left')
    
plt.show()

In [ ]:
# [예제 3.38] 변화율로 살펴보는 그래프 

plt.figure(figsize=(10,6))
ax = plt.subplot()
ax2 = ax.twinx()

si = '서울'
gu = '서울'

plt.title(si + '-' + gu)
ln1 = ax.plot(price_index[si][gu]['2009-1':].pct_change(12), label='매매가')
ln2 = ax.plot(jun_index[si][gu]['2009-1':].pct_change(12), label='전세가')
ln3 = ax2.plot(modified_permission[si]['2009':].pct_change(), label='인허가', color='lightslategray', ls='--')
ln4 = ax2.plot(unsold[si]['2009':].pct_change(12), label='미분양', color='y', ls='--')
lns = ln1 +ln2 + ln3 + ln4
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc='upper left')
    
plt.show()

In [ ]:
# [예제 3.39] demand 함수를 실행해 결과 저장 

from datetime import datetime
from dateutil.relativedelta import relativedelta

def demand(price_index, jeonse_index, index_date, time_range):

    prev_date = index_date - relativedelta(months=time_range)
    prev_date2 = index_date - relativedelta(months=time_range*3)

    demand_df = pd.DataFrame()
    demand_df['매매증감률'] = (price_index.loc[index_date] - price_index.loc[prev_date])/ price_index.loc[prev_date].replace(0,None)
    demand_df['전세증감률'] = (jeonse_index.loc[index_date] - jeonse_index.loc[prev_date])/jeonse_index.loc[prev_date].replace(0,None)
    demand_df['이전최대값'] = price_index[prev_date2:index_date][:-1].max()
    demand_df['최댓값대비증감률'] = (price_index.loc[index_date] - demand_df['이전최대값']) /demand_df['이전최대값'].replace(0,None)

    demand_df['매매가상승'] = demand_df['매매증감률'] > 0.01
    demand_df['전세가상승'] = demand_df['전세증감률'] > 0.01
    demand_df['더빠른전세상승'] = demand_df['전세증감률'] > demand_df['매매증감률']
    demand_df['최댓값대비상승'] = demand_df['최댓값대비증감률'] > 0
    demand_df['수요총합'] = demand_df[['매매가상승','전세가상승','더빠른전세상승','최댓값대비상승']].sum(axis=1)

    demand_df = demand_df[demand_df['수요총합'] == 4]

    seleted_index = []

    for name in demand_df.index:
        if name[0] is not name[1]:
            seleted_index.append((name[0], name[1]))

    demand_df = demand_df.loc[seleted_index]
    
    return demand_df


index_date = datetime(2013, 1, 1)
time_range = 12
demand_1 = demand(price_index, jun_index, index_date, time_range)

In [ ]:
# [예제 3.40] demand 함수 결과를 인허가, 미분양 데이터와 함께 보기

prev_date = index_date - relativedelta(months=time_range)
prev_date2 = index_date - relativedelta(months=time_range * 3)
graph_start = index_date - relativedelta(months=time_range * 3)

num_row = int((len(demand_1.index)-1)/2)+1

plt.figure(figsize=(15, num_row*5))
for i, spot in enumerate(demand_1.index):
    ax = plt.subplot(num_row, 2, i+1)
    si = spot[0]
    gu = spot[1]
    plt.title(spot)
    ax2 = ax.twinx()
    ln1 = ax.plot(price_index[si][gu][graph_start:], label='매매가')
    ln2 = ax.plot(jun_index[si][gu][graph_start:], label='전세가')
    ln3 = ax2.plot(modified_permission[si][graph_start:]/10,  color='lightslategray', label='인허가')
    ln4 = ax2.plot(unsold[si][graph_start:], color='y', label='미분양')
    ax.axvline(x=index_date, color='lightcoral', linestyle='--')
    ax.axvline(x=prev_date, color='darkseagreen', linestyle='--')
    ax.axvline(x=prev_date2, color='darkseagreen', linestyle='--')
    lns = ln1 +ln2 + ln3 + ln4
    labs = [l.get_label() for l in lns]
    ax.legend(lns, labs, loc='lower right')

plt.show()
